In [63]:
import pandas as pd
import yfinance as yf
from yahoo_fin import stock_info as si
import datetime as dt

# para ignorar uns warnings chatos do Jupyter
import warnings
warnings.filterwarnings('ignore')

In [28]:
# Honestamente não sei o quão atualizado é esta lista de tickers do yahoo
tickers = si.tickers_ibovespa()

In [29]:
tickers_sa = []
for ticker in tickers:
    tickers_sa.append(f'{ticker}.SA')
    

In [30]:
start = dt.datetime.now() - dt.timedelta(days=365)
end = dt.datetime.now()

In [31]:
ibovespa = yf.download('^BVSP',start,end)
ibovespa['Pct Change']=ibovespa['Adj Close'].pct_change()
ibovespa['Return']=(ibovespa['Pct Change']+1).cumprod()
ibovespa_return = ibovespa['Return'][-1]

[*********************100%***********************]  1 of 1 completed


In [34]:
return_list = []
for ticker in tickers_sa:
    try:
        df = yf.download(ticker,start,end)
        df['Pct Change']=df['Adj Close'].pct_change()
        df['Return']=(df['Pct Change']+1).cumprod()
        stock_return = df['Return'][-1]
        stock_return_compared = round((stock_return/ibovespa_return),2)
        return_list.append(stock_return_compared)
    except:
        return_list.append(0.00)
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BRDT3.SA: No timezone found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***

In [81]:
best_performers = pd.DataFrame(list(zip(tickers_sa,return_list)),columns=['Ticker','Returns Compared'])
best_performers['Score']=best_performers['Returns Compared'].rank(pct=True)*100
best_performers.sort_values(by=['Score'],ascending=False,inplace=True)
top_10 = best_performers.head(10)
top_10

,Ticker,Returns Compared,Score
25,CYRE3.SA,1.45,100.000000
14,BRSR6.SA,1.44,98.684211
57,PETR4.SA,1.43,97.368421
56,PETR3.SA,1.40,96.052632
30,EMBR3.SA,1.39,94.078947
1,AZUL4.SA,1.39,94.078947
3,BBAS3.SA,1.36,91.447368
69,UGPA3.SA,1.36,91.447368
38,GOLL4.SA,1.33,89.473684
59,RADL3.SA,1.26,87.500000


In [82]:
final_df = pd.DataFrame(columns=['Ticker','Preço','Min 52 semanas','Max 52 semanas','PL','LPA','Market Cap'])

In [85]:
for ticker in top_10['Ticker']:
    latest_price = si.get_quote_table(ticker)['Previous Close']
    PL = si.get_quote_table(ticker)['PE Ratio (TTM)']
    LPA = si.get_quote_table(ticker)['EPS (TTM)']
    market_cap = si.get_quote_table(ticker)['Market Cap']
    
    df = yf.download(ticker,start,end)
    low_52week = round(min(df['Low'][-(52*5):]),2)
    high_52week = round(max(df['High'][-(52*5):]),2)
    
    final_df = final_df.append({'Ticker':ticker,
                    'Preço':latest_price,
                    'Min 52 semanas':low_52week,
                    'Max 52 semanas':high_52week,
                    'PL':PL,
                    'LPA':LPA,
                    'Market Cap':market_cap},ignore_index=True)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [87]:
final_df

,Ticker,Preço,Min 52 semanas,Max 52 semanas,PL,LPA,Market Cap
0,CYRE3.SA,19.91,6.48,21.51,8.77,2.40,7.892B
1,BRSR6.SA,14.90,6.48,21.51,6.29,2.40,6.117B
2,PETR4.SA,29.65,6.48,21.51,2.24,13.15,411.82B
3,PETR3.SA,33.31,6.48,21.51,2.51,13.18,411.819B
4,EMBR3.SA,18.60,6.48,21.51,NaN,-1.25,13.392B
5,AZUL4.SA,19.14,6.48,21.51,NaN,-2.07,6.71B
6,BBAS3.SA,49.01,6.48,21.51,3.86,12.93,142.575B
7,UGPA3.SA,18.45,6.48,21.51,11.84,1.63,21.027B
8,GOLL4.SA,11.90,6.48,21.51,NaN,-0.25,38.399B
9,RADL3.SA,28.05,6.48,21.51,46.70,0.61,48.843B
